In [1]:
import pandas as pd 
import numpy as np 

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression , RidgeClassifier, Huber
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier , GradientBoostingClassifier , ExtraTreesClassifier , AdaBoostClassifier
from sklearn.model_selection import KFold ,StratifiedKFold
from sklearn.metrics import log_loss
import warnings 
warnings.filterwarnings('ignore')

In [3]:
train = pd.read_csv('Train.csv')
test = pd.read_csv('Test.csv')

In [4]:
train = train[train['time']<2*1e6]
train = train[train['time']!=1804691]

train.head()

,Store_Ratio,Basket_Ratio,Category_1,Store_Score,Category_2,Store_Presence,Score_1,Score_2,Score_3,Score_4,time,popularity
0,0.407,0.00380,2,-35.865,1,0.9920,0.944,0.0988,0.1100,113.911,189125,4
1,0.234,0.10500,0,-19.884,1,0.9440,0.900,0.1290,0.0382,76.332,186513,4
2,0.668,0.72600,9,-7.512,1,0.0387,0.000,0.1340,0.4530,124.075,172143,4
3,0.184,0.00561,4,-34.357,1,0.8490,0.931,0.1110,0.0641,79.037,184000,5
4,0.231,0.13100,6,-22.842,1,0.9370,0.000,0.1090,0.0677,109.560,186507,3


In [5]:
X = train.drop('popularity',axis=1)
Y = train['popularity']

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_ = scaler.fit_transform(X)
X = pd.DataFrame(X_,columns=X.columns)

folds = StratifiedKFold(n_splits=10,shuffle=True ,random_state=42)

oof_preds = []
oof_scores = []

for i , (train_idx,test_idx) in enumerate(folds.split(X,Y)):
  train_set = (X.iloc[train_idx],Y.iloc[train_idx])
  test_set = (X.iloc[test_idx],Y.iloc[test_idx])

  model = XGBClassifier()
  model.fit(*train_set)

  preds = model.predict_proba(test_set[0])
  score = log_loss(test_set[1],preds)
  
  oof_pred_fold = model.predict_proba(scaler.transform(test))
  oof_preds.append(oof_pred_fold)

  print(score)
  oof_scores.append(score)

print("Mean OOF Score :",np.mean(oof_scores))
final_preds = np.mean(np.array(oof_preds),axis=0)

0.4305184737109604
0.41766059063992234
0.43085249635628564
0.4234314244132961
0.4044471588680666
0.39137786328650254
0.38624618443323666
0.4062241207144625
0.40654886399938983
0.3988892405018195
Mean OOF Score : 0.4096196416923942


In [7]:
submission = pd.DataFrame(final_preds)
submission.to_csv("submission_2301.csv",index=None)
submission

In [16]:
dummy = test[test['Store_Ratio']==0]
dummy.index

Int64Index([358, 2873, 5982, 5984, 6306, 7565, 7761, 8615, 8829, 11152, 12010], dtype='int64')

In [9]:
# submission.iloc[dummy.index]

In [10]:
# for i in dummy.index:
#     submission.loc[i] = [1.0,0.0,0.0,0.0,0.0]

In [11]:
# # submission = pd.DataFrame(final_preds)
# submission.to_csv("submission_2301.csv",index=None)
# submission

In [14]:
X = train.drop('popularity',axis=1)
Y = train['popularity']

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_ = scaler.fit_transform(X)
X = pd.DataFrame(X_,columns=X.columns)

folds = StratifiedKFold(n_splits=10,shuffle=True ,random_state=42)

oof_preds = []
oof_scores = []

for i , (train_idx,test_idx) in enumerate(folds.split(X,Y)):
  train_set = (X.iloc[train_idx],Y.iloc[train_idx])
  test_set = (X.iloc[test_idx],Y.iloc[test_idx])

  model = CatBoostClassifier(loss_function='MultiClass', 
                         eval_metric='TotalF1', 
                         depth=7,
                         verbose=0
)
  model.fit(*train_set)
  preds = model.predict_proba(test_set[0])
  score = log_loss(test_set[1],preds)
  
  oof_pred_fold = model.predict_proba(scaler.transform(test))
  oof_preds.append(oof_pred_fold)

  print(score)
  oof_scores.append(score)

print("Mean OOF Score :",np.mean(oof_scores))
final_preds = np.mean(np.array(oof_preds),axis=0)

0.4043593882922242
0.4099553297858104
0.4145086229270212
0.4174290309787979
0.3975906794488943
0.38083747477598145
0.37915732472136604
0.40758358589353066
0.38997410575482006
0.39756641737723
Mean OOF Score : 0.3998961959955676


In [17]:
submission = pd.DataFrame(final_preds)
for i in dummy.index:
    submission.loc[i] = [1.0,0.0,0.0,0.0,0.0]
submission.to_csv("submission_2301_cat.csv",index=None)
    

In [20]:
train.popularity.value_counts()

4    15229
3     2165
5      472
1      323
0       16
Name: popularity, dtype: int64

In [21]:
from imblearn.under_sampling import RandomUnderSampler 


In [37]:
train.popularity.value_counts()

4    15229
3     2165
5      472
1      323
0       16
Name: popularity, dtype: int64

In [34]:
X.shape,Y.shape

((2992, 11), (2992,))

In [42]:
dum1 = train[train['popularity']==4]
dum2 = train[train['popularity']!=4]

In [43]:
dum1 = dum1.sample(frac=0.75)
dum1.shape


(11422, 12)

In [44]:
dummy_train = dum1.append(dum2)
dummy_train.popularity.value_counts()

4    11422
3     2165
5      472
1      323
0       16
Name: popularity, dtype: int64

In [1]:
# X = dummy_train.drop('popularity',axis=1)
# Y = dummy_train['popularity']

# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()
# X_ = scaler.fit_transform(X)
# X = pd.DataFrame(X_,columns=X.columns)

# folds = StratifiedKFold(n_splits=10,shuffle=True ,random_state=42)

# oof_preds = []
# oof_scores = []

# for i , (train_idx,test_idx) in enumerate(folds.split(X,Y)):
#   train_set = (X.iloc[train_idx],Y.iloc[train_idx])
#   test_set = (X.iloc[test_idx],Y.iloc[test_idx])

#   model = CatBoostClassifier(loss_function='MultiClass', 
#                          eval_metric='TotalF1', 
#                          depth=7,
#                          verbose=0
# )
#   model.fit(*train_set)
#   preds = model.predict_proba(test_set[0])
#   score = log_loss(test_set[1],preds)
  
#   oof_pred_fold = model.predict_proba(scaler.transform(test))
#   oof_preds.append(oof_pred_fold)

#   print(score)
#   oof_scores.append(score)

# print("Mean OOF Score :",np.mean(oof_scores))
# final_preds = np.mean(np.array(oof_preds),axis=0)